In [ ]:
import numpy as np

from db.db_utils import (
    get_all_leagues, get_all_heroes, get_all_players, get_active_players_by_team,
    get_relevant_matches
)
from db.models import Match

In [ ]:
team_id = 7119388
team_players = get_active_players_by_team(team_id)
[player.player_name for player in team_players]

In [ ]:
def calculate_match_stats(matches: list[Match]) -> dict:
    last_hits_at_5 = [match.last_hits_at_5 for match in matches]
    # denies_at_5 = [match.denies_at_5 for match in matches]
    # kills = [match.kills for match in matches]
    
    if len(last_hits_at_5) == 0:
        return {}

    return {
        "count": len(last_hits_at_5),
        "mean": np.mean(last_hits_at_5),
        "median": np.median(last_hits_at_5),
        "std": np.std(last_hits_at_5),
        "min": np.min(last_hits_at_5),
        "max": np.max(last_hits_at_5),
        "q25": np.percentile(last_hits_at_5, 25),
        "q75": np.percentile(last_hits_at_5, 75),
    }

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display

all_option = ('All', None)
league_info = [(league.league_name, league.league_id) for league in get_all_leagues()]
hero_info = [(hero.hero_name, hero.hero_id) for hero in get_all_heroes()]
hero_name_info = [(hero.hero_name, hero.hero_name) for hero in get_all_heroes()]
# team_info = [(team.team_name, team.team_id) for team in get_all_teams()]
player_info = [(player.player_name, player.player_account_id) for player in get_all_players()]

# Create widgets for Plotly
league_dropdown = widgets.Dropdown(
    options=[all_option] + league_info,
    description='League:'
)

player_dropdown = widgets.Dropdown(
    options=[all_option] + player_info,
    description='Player:'
)

player_hero_dropdown = widgets.Dropdown(
    options=[all_option] + hero_info,
    description='Player hero:'
)

opponent_hero_dropdown = widgets.Dropdown(
    options=[all_option] + hero_name_info,
    description='Opponent hero:'
)

# team_dropdown = widgets.Dropdown(
#     options=[all_option] + team_info,
#    description='Team:'
#)


# Output widget for Plotly figure
plotly_output = widgets.Output()

In [ ]:
def update_plotly_plot(league_id, player_id, player_hero, opponent_hero):
    with plotly_output:
        plotly_output.clear_output(wait=True)
        
        # Get data
        matches = get_relevant_matches(
            league_id=league_id, player_id=player_id,
            hero_id=player_hero, opponent_hero_name=opponent_hero
        )
        data = [match.last_hits_at_5 for match in matches]
        
        lh_stats = calculate_match_stats(matches)
        print(lh_stats)
        
        # Create Plotly histogram
        fig = go.Figure()
        
        fig.add_trace(go.Histogram(
            x=data,
            nbinsx=80,
            name='Last Hits at 5',
            hovertemplate='<b>Last Hits:</b> %{x}<br><b>Count:</b> %{y}<extra></extra>'
        ))
        
        fig.update_layout(
            title=f'Distribution of Last Hits at 5 (n={len(data)})',
            xaxis_title='Last Hits at 5',
            yaxis_title='Frequency',
            yaxis_type='linear',
            height=500
        )
        
        fig.show()

# Create interactive Plotly plot
plotly_interactive = widgets.interactive(
    update_plotly_plot,
    league_id=league_dropdown,
    player_id=player_dropdown,
    player_hero=player_hero_dropdown,
    opponent_hero=opponent_hero_dropdown,
)

In [ ]:
display(plotly_interactive, plotly_output)

In [ ]:
# show the games

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np

def create_enhanced_plotly_plot(league_id, player_id, bins, log_scale):
    # Get data
    matches = get_relevant_matches(league_id=league_id, player_id=player_id)
    data = [match.last_hits_at_5 for match in matches]
    
    # Calculate statistics
    stats = calculate_match_stats(matches)
    
    # Create main histogram
    fig = go.Figure()
    
    fig.add_trace(go.Histogram(
        x=data,
        nbinsx=bins,
        name='Last Hits at 5',
        hovertemplate='<b>Last Hits:</b> %{x}<br><b>Count:</b> %{y}<extra></extra>',
        marker_color='lightblue',
        opacity=0.7
    ))
    
    # Add vertical lines for key statistics
    # Mean line
    fig.add_vline(
        x=stats['mean'], 
        line_dash="dash", 
        line_color="red",
        annotation_text=f"Mean: {stats['mean']:.2f}",
        annotation_position="top"
    )
    
    # Median line
    fig.add_vline(
        x=stats['median'], 
        line_dash="dot", 
        line_color="green",
        annotation_text=f"Median: {stats['median']:.1f}",
        annotation_position="top"
    )
    
    # Q25 and Q75 lines
    fig.add_vline(
        x=stats['q25'], 
        line_dash="dashdot", 
        line_color="orange",
        annotation_text=f"Q25: {stats['q25']:.1f}",
        annotation_position="bottom"
    )
    
    fig.add_vline(
        x=stats['q75'], 
        line_dash="dashdot", 
        line_color="orange",
        annotation_text=f"Q75: {stats['q75']:.1f}",
        annotation_position="bottom"
    )
    
    # Add shaded area for standard deviation around mean
    std_lower = stats['mean'] - stats['std']
    std_upper = stats['mean'] + stats['std']
    
    fig.add_vrect(
        x0=std_lower, 
        x1=std_upper,
        fillcolor="red", 
        opacity=0.1,
        line_width=0,
        annotation_text=f"±1σ: {stats['std']:.2f}",
        annotation_position="top right"
    )
    
    # Update layout with statistics in title and annotations
    fig.update_layout(
        title=f'Distribution of Last Hits at 5<br><sub>n={stats["count"]} | Mean: {stats["mean"]:.2f} | Std: {stats["std"]:.2f} | Range: {stats["min"]}-{stats["max"]}</sub>',
        xaxis_title='Last Hits at 5',
        yaxis_title='Frequency',
        yaxis_type='log' if log_scale else 'linear',
        height=600,
        showlegend=False,
        # Add text box with all statistics
        annotations=[
            dict(
                x=0.02, y=0.98,
                xref='paper', yref='paper',
                text=f'''<b>Statistics:</b><br>
                Count: {stats["count"]}<br>
                Mean: {stats["mean"]:.2f}<br>
                Median: {stats["median"]:.1f}<br>
                Std Dev: {stats["std"]:.2f}<br>
                Min: {stats["min"]}<br>
                Max: {stats["max"]}<br>
                Q25: {stats["q25"]:.1f}<br>
                Q75: {stats["q75"]:.1f}''',
                showarrow=False,
                align='left',
                bgcolor='rgba(255,255,255,0.8)',
                bordercolor='black',
                borderwidth=1,
                font=dict(size=10)
            )
        ]
    )
    
    return fig

# Use this in your interactive widget
def update_plotly_plot(league_id, player_id, bins, log_scale):
    with plotly_output:
        plotly_output.clear_output(wait=True)
        fig = create_enhanced_plotly_plot(league_id, player_id, bins, log_scale)
        fig.show()